[current dir](.)

In [ ]:
# immune_system_input = """17 5390 weak radiation bludgeoning;4507 fire 2
# 989 1274 immune fire weak bludgeoning slashing;25 slashing 3"""
# infection_input = """801 4706 weak radiation;116 bludgeoning 1
# 4485 2961 immune radiation weak fire cold;12 slashing 4"""

immune_system_input = """2785 4474 weak cold;14 fire 20
4674 7617 immune slashing bludgeoning weak fire;15 slashing 15
1242 1934 weak fire;15 bludgeoning 6
1851 9504 weak bludgeoning;47 slashing 2
846 9124 weak bludgeoning immune radiation;99 bludgeoning 4
338 1378 immune radiation;39 cold 10
3308 5087 weak radiation;12 fire 3
2668 8316 weak bludgeoning radiation;28 slashing 9
809 1756 immune bludgeoning;21 cold 1
4190 8086 immune cold;18 cold 5"""
infection_input = """2702 10159;7 fire 7
73 14036 weak fire;384 radiation 18
4353 35187;15 slashing 14
370 9506 weak bludgeoning radiation;46 slashing 12
4002 22582 weak radiation cold;11 fire 8
1986 24120 immune fire;22 radiation 11
1054 17806;25 cold 16
124 37637;589 cold 19
869 11019 weak fire;24 cold 17
3840 38666 immune slashing fire bludgeoning;19 bludgeoning 13"""

In [ ]:
class Group:
    def __init__(self, side, line, boost=0):
        self.side = side

        attribs, attack = line.split(';')
        units, hp, *type_mods = attribs.split()
        units=int(units)
        hp=int(hp)
        weak = []
        immune = []
        cur = None
        for w in type_mods:
            if w == "weak":
                cur = weak
            elif w == "immune":
                cur = immune
            else:
                cur.append(w)

        self.units = units
        self.hp = hp
        self.weak = weak
        self.immune = immune

        attack_amount, attack_type, initiative = attack.split()
        attack_amount = int(attack_amount)
        initiative = int(initiative)

        self.attack = attack_amount + boost
        self.attack_type = attack_type
        self.initiative = initiative

        self.attacker = None
        self.target = None

    def clear(self):
        self.attacker = None
        self.target = None

    def choose(self, groups):
        assert self.target is None
        cands = [group for group in groups
                if group.side != self.side
                and group.attacker is None
                and self.damage_prio(group)[0] > 0]
        if cands:
            self.target = max(cands, key=lambda group: self.damage_prio(group))
            assert self.target.attacker is None
            self.target.attacker = self

    def effective_power(self):
        return self.units * self.attack

    def target_prio(self):
        return (-self.effective_power(), -self.initiative)

    def damage_prio(self, target):
        if target.units == 0:
            return (0, 0, 0)
        if self.attack_type in target.immune:
            return (0, 0, 0)
        mul = 1
        if self.attack_type in target.weak:
            mul = 2
        return (mul * self.units * self.attack, target.effective_power(), target.initiative)

    def do_attack(self, target):
        total_attack = self.damage_prio(target)[0]
        killed = total_attack // target.hp
        target.units = max(0, target.units - killed)

In [ ]:
def solve(boost):
    immune_system_groups = [Group(False, line, boost) for line in immune_system_input.split("\n")]
    infection_groups = [Group(True, line) for line in infection_input.split("\n")]
    
    groups = immune_system_groups + infection_groups

    old = (-1, -1)
    while True:
        groups = sorted(groups, key=lambda group: group.target_prio())
        for group in groups:
            group.clear()
        for group in groups:
            group.choose(groups)
        groups = sorted(groups, key=lambda group: -group.initiative)
        for group in groups:
            if group.target:
                group.do_attack(group.target)

        immune_system_units = sum(group.units for group in groups if group.side == False)
        infection_units = sum(group.units for group in groups if group.side == True)
        if (immune_system_units, infection_units) == old:
            return (immune_system_units, infection_units)
        old = (immune_system_units, infection_units)

Part 1

In [ ]:
def part1():
    return solve(0)[1]

part1()

Part 2

In [ ]:
def part2():
    for boost in range(1000000):
        ans = solve(boost)
        if ans[1] == 0:
            return ans[0]
        
part2()